In [ ]:
#importing libraries
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import string
import nltk
import pattern
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
import contractions
from collections import  Counter
import emot
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from matplotlib import rcParams
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import unicodedata
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Creating Query Search
base_url="https://www.amazon.in/s?k="
search_query="sports+shoes&crid=3D9RT0WJS6QZC&sprefix=sports+shoes%2Caps%2C287&ref=nb_sb_noss_1"
url=base_url+search_query

# Setting header for the scraper
header={
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/77.0.3865.90 Safari/537.36'}

# Getting the html data
search_response=requests.get(url,headers=header)

# Checking response status
search_response.status_code

# Getting the html data
search_response.text

# Using BeautifulSoup to parse the data
soup = BeautifulSoup(search_response.content)
data_asin = []

# Finding ASIN product codes
for i in soup.find_all('div',
                    class_= 'sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20'):
    data_asin.append(i["data-asin"])
    


In [ ]:
# Function to search ASIN
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [ ]:
# Creating a list to save the links of the pages where the reviews are
link=[]
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

In [ ]:
# Function to search reviews using ASIN
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"    

In [ ]:
# Creating 4 lists to scrape the required data
reviews=[]
star=[]
title=[]
name=[]
for j in range(len(link)):
    for k in range(100):
        response=Searchreviews(link[j]+'&pageNumber='+str(k))
        
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text)
        for i in soup.findAll("i",{'data-hook':'review-star-rating'}):
            star.append(i.text)
        for i in soup.findAll("a",{'data-hook':'review-title'}):
            title.append(i.text)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            name.append("https://www.amazon.in"+link[j])

In [ ]:
# Creating a dataframe and exporting as a csv file for analysis
df1 = pd.DataFrame({'name':name,'title':title,'reviews':reviews,'star':star})
df1.to_csv("Amazon_Reviews.csv")

In [ ]:
# Read the Data
data = pd.read_csv('C:/Users/malay/Amazon reviews.csv')
data.sample(10)

In [ ]:
# Creating a copy of data
df = data.copy()

# Checking for null values and dropping them
df.isnull().sum()
df = df.dropna()

In [ ]:
# Converting reviews to string datatype
df['reviews'] = df['reviews'].astype(str)

# Dropping Duplicates
df = df.drop_duplicates(keep = 'first')

# Removing '\n' present in reviews
df['reviews']=df['reviews'].apply(lambda x:x.strip('\n'))
df['title']=df['title'].apply(lambda x:x.strip('\n'))

# Removing strings that are not required
df['fin_review'] = df['fin_review'].apply(lambda x : x.replace("The media could not be loaded",""))
df['fin_review'] = df['fin_review'].apply(lambda x : x.replace("nan",""))

# Merging title and content of reviews
df['fin_review'] = df['title'] + ' ' + df['reviews']

# Getting the name of the brand from URL
df['name'] = df['name'].apply(lambda x : x.replace("https://www.amazon.in/",""))
df['name'] = df['name'].apply(lambda x:x.partition('-')[0])

# Getting the star value
df['star'] = df['star'].apply(lambda x:x[0:3])

In [ ]:
# Function for converting emojis into word
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text
df['fin_review'] = df['fin_review'].apply(lambda x : convert_emojis(x))

In [ ]:
# Function to get POS Tagging
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
# Function to clean the data(stopwords, punctuations,lowercase,contractions)
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    text = [x for x in text if x not in STOPWORDS]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)
df['clean_fin_review'] = df['fin_review'].apply(lambda x: clean_text(x))

In [ ]:
# Initiating the VADER module
analyser = SentimentIntensityAnalyzer()

# Function to calculate sentiment score
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

# Function to calculate compound score
def compound_score(text):
    comp=sentiment_analyzer_scores(text)
    return comp['compound'] # returns the compound score from the dictionary

# Applying on the reviews column to get the score
df['sentiment_score']=df['clean_fin_review'].apply(lambda x:compound_score(x)) 

# Function to classify category for reviews
def sentiment_category(score):
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"
df['review_category']=df['sentiment_score'].apply(lambda x:sentiment_category(x))    

In [ ]:
# Function to return words with the highest frequencies
def getMostCommon(reviews_list,topn=20):
    reviews=" ".join(reviews_list)
    tokenised_reviews=reviews.split(" ")
    
    
    freq_counter=Counter(tokenised_reviews)
    return freq_counter.most_common(topn) 

# Function to plot the most common top words in reviews
def plotMostCommonWords(reviews_list,topn=20,title="Common Review Words",color="mediumseagreen"): 
    top_words=getMostCommon(reviews_list,topn=topn)
    data=pd.DataFrame()
    data['words']=[val[0] for val in top_words]
    data['freq']=[val[1] for val in top_words]
    ax = sns.barplot(y='words',x='freq',data=data,color=color).set_title(title+" top "+str(topn))
    plt.show()
    
    

# Function to generate n-grams
def generateNGram(text,n):
    tokens=text.split(" ")
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return ["_".join(ngram) for ngram in ngrams]
reviews_bigrams=[" ".join(generateNGram(i,2)) for i in df['clean_fin_review']]
plotMostCommonWords(reviews_bigrams,20,"Review Bigrams")

In [ ]:
# Converting data to csv for further analysis in PowerBi
df.to_csv("Amazon Reviews Data.csv")

In [ ]:
# Read the data for Twitter Analysis 
data = pd.read_csv('C:/Users/malay/Downloads/archive (7)/twitter_training.csv')

In [ ]:
# make a copy of data
df = data.copy()

# Drop irrelevant and neutral sentiment
df.drop(df[df['Sentiment'] == 'Irrelevant'].index, inplace = True)
df.drop(df[df['Sentiment'] == 'Neutral'].index, inplace = True)

# Drop Duplicates
df1 = df.drop_duplicates(keep = 'first')

# Drop Missing values
df2 = df1.dropna()

In [ ]:
# Convert the tweets to string
df2['tweets'] = df2['tweets'].astype(str)

# Add a length of tweets column
df2['tweet_length'] = df2['tweets'].str.len()

# Remove HTML tags
df2['tweets']=df2['tweets'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())

# Remove URL's
df2['tweets']=df2['tweets'].apply(lambda x: re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+
                                    [.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+
                                    (?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "",x))

# Remove E-mails
df2['tweets']=df2['tweets'].apply(lambda x: re.sub(r'([a-zA-Z0-9+_-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)','',x))

In [ ]:
# Function to remove accentuated characters
def remove_acc_chars(x):
    x=unicodedata.normalize('NFKD',x).encode('ascii','ignore').decode('utf-8','ignore')
    return x
df2['tweets']=df2['tweets'].apply(lambda x: remove_acc_chars(x))


In [ ]:
# Applying the clean text function from above 
df2['clean_tweets'] = df2['tweets'].apply(lambda x: clean_text(x))

In [ ]:
# Converting data to csv for further analysis in PowerBi 
df2.to_csv("Twitter Data.csv")